In [68]:
import pandas as pd
from dotenv import load_dotenv
import os
import time
load_dotenv()

True

# OPM Employment Data
FedScope Employment Cube (March 2024)  
from  
https://www.opm.gov/data/datasets/

In [69]:
file_path = "../datasets/OPM/FedScope Employment Cube (March 2024)/FACTDATA_MAR2024.TXT"
fed_employment = pd.read_csv(file_path, sep=",", engine="python", on_bad_lines="skip", encoding="utf-8")
fed_employment.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,SALLVL,STEMOCC,SUPERVIS,TOA,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS
0,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,20,XXXX,2,50,F,1,202403,1,210000.0,20.8
1,AA00,11,J,15,NaN,I,0905,1,ES,ES-**,20,XXXX,2,50,F,1,202403,1,203000.0,31.2
2,AA00,11,K,04,NaN,G,0301,2,99,EX-02,30,XXXX,2,48,F,2,202403,1,NaN,22.0
3,AA00,11,C,04,12,B,0560,2,99,GS-12,9,XXXX,8,15,F,1,202403,1,99200.0,2.5
4,AA00,11,D,04,13,B,0905,1,99,GS-13,11,XXXX,8,30,F,1,202403,1,117962.0,2.5


# OPM Agency by Code

from  
https://dw.opm.gov/datastandards/referenceData/1388/current?sort=EXPLANATION&page=1&dir=desc

In [70]:
# Extract those codes from a messy text file.
# https://acf.gov/sites/default/files/documents/paris/federal_codes_for_states_03212018.pdf
def clean_agency_name(agency_name):
    name = agency_name.replace("Note: Organization does not make HR submissions to OPM. Code may be used for other documentation purposes.", "")
    name = name.strip()
    name = " ".join(name.split())
    return name
    
agency_codes = pd.read_csv("../datasets/OPM/Misc/agency_code_by_id_full.csv")
agency_codes['Agency'] = agency_codes.apply(lambda row: clean_agency_name(row['Agency']), axis=1)
agency_codes.head()

,Code,Agency,From,Through
0,WX00,World War 1 Centennial Commission,6/13/2014,Present
1,SM02,Woodrow Wilson International Center for Scholars,5/1/1985,Present
2,DLWB,Women's Bureau,6/1/1998,Present
3,WH01,White House Office.,5/1/1983,Present
4,GN00,White House Commission on the National Moment ...,1/1/2002,Present


# Budget by Agency

In [71]:
with open("../datasets/OPM/Misc/budget_by_agency.html", 'r') as f:
    raw_table = f.read()

budgets = pd.read_html(raw_table)[0]

### Light cleaning ###
budgets.rename(columns={
    "Agency Name  Sort table by ascending Agency Name  Sort table by descending Agency Name": "Agency",
    "Budgetary Resources  Sort table by ascending Budgetary Resources  Sort table by descending Budgetary Resources": "Budget",
    "Percent of Total  Sort table by ascending Percent of Total  Sort table by descending Percent of Total": "Percent of Total Budget"
}, inplace=True)

budgets['Agency'] = budgets.apply(lambda row: clean_agency_name(row['Agency']), axis=1)

# If math needed on dollar values, uncomment.
# def clean_money(dollars: str) -> str:
#     dollars = dollars.replace('\r', '')
#     dollars = dollars.replace('$', '')
#     dollars = dollars.replace(',', '')
#     return float(dollars)
    

# budgets['Budget'] = budgets.apply(lambda row: clean_money(row['Budget']), axis=1)

budgets.head()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_90420\2015976124.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  budgets = pd.read_html(raw_table)[0]


,Agency,Congressional Justification of Budget (CJ),Budget,Percent of Total Budget
0,Department of the Treasury (TREAS),https://www.treasury.gov/cj,"$3,125,545,544,294",28.86%
1,Department of Health and Human Services (HHS),https://www.hhs.gov/cj,"$2,369,129,444,329",21.88%
2,Department of Defense (DOD),https://www.defense.gov/cj,"$1,713,980,313,483",15.83%
3,Social Security Administration (SSA),https://www.ssa.gov/cj,"$452,144,957,484",4.17%
4,Department of Veterans Affairs (VA),https://www.va.gov/cj,"$407,272,934,513",3.76%


# Agency by Department

This was derived from an LLM classifying each agency.

In [72]:
agency_to_department = pd.read_csv("../datasets/CUSTOM/agency_to_department.csv")
agency_to_department['Agency'] = agency_to_department.apply(lambda row: clean_agency_name(row['Agency']), axis=1)
agency_to_department['Department'] = agency_to_department.apply(lambda row: clean_agency_name(row['Department']), axis=1)
agency_to_department.head()

,Agency,Department
0,Administrative Conference of the United States,Independent Agency
1,American Battle Monuments Commission,Independent Agency
2,Air Force Inspection Agency (FO),Department of Defense (DOD)
3,Air Force Operational Test and Evaluation Center,Department of Defense (DOD)
4,Air Force Audit Agency,Department of Defense (DOD)


# Combination of Data

We want to see the number of employees, and budget for each department.  

Step 1: Join the employment data with the agency data to get the names of the agencies.  
Step 2: Join the agency by department table, to add on the appropriate department name.  
Step 3: Group by Department to gather the number of employees per department.  
Step 4: Join the Budget by department table, to see budget for each agency.

In [73]:
# Step 1
fed_employment = fed_employment.merge(agency_codes, left_on='AGYSUB', right_on='Code', how='left')
fed_employment['Agency'] = fed_employment['Agency'].fillna("UNKNOWN")
fed_employment.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,WORKSCH,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS,Code,Agency,From,Through
0,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,...,F,1,202403,1,210000.0,20.8,AA00,Administrative Conference of the United States,3/1/2010,Present
1,AA00,11,J,15,NaN,I,0905,1,ES,ES-**,...,F,1,202403,1,203000.0,31.2,AA00,Administrative Conference of the United States,3/1/2010,Present
2,AA00,11,K,04,NaN,G,0301,2,99,EX-02,...,F,2,202403,1,NaN,22.0,AA00,Administrative Conference of the United States,3/1/2010,Present
3,AA00,11,C,04,12,B,0560,2,99,GS-12,...,F,1,202403,1,99200.0,2.5,AA00,Administrative Conference of the United States,3/1/2010,Present
4,AA00,11,D,04,13,B,0905,1,99,GS-13,...,F,1,202403,1,117962.0,2.5,AA00,Administrative Conference of the United States,3/1/2010,Present


In [74]:
# Step 2
fed_employment = fed_employment.merge(agency_to_department, on='Agency', how='left')
fed_employment.head()

,AGYSUB,LOC,AGELVL,EDLVL,GSEGRD,LOSLVL,OCC,PATCO,PP,PPGRD,...,WORKSTAT,DATECODE,EMPLOYMENT,SALARY,LOS,Code,Agency,From,Through,Department
0,AA00,11,F,13,NaN,G,0340,2,ES,ES-**,...,1,202403,1,210000.0,20.8,AA00,Administrative Conference of the United States,3/1/2010,Present,Independent Agency
1,AA00,11,J,15,NaN,I,0905,1,ES,ES-**,...,1,202403,1,203000.0,31.2,AA00,Administrative Conference of the United States,3/1/2010,Present,Independent Agency
2,AA00,11,K,04,NaN,G,0301,2,99,EX-02,...,2,202403,1,NaN,22.0,AA00,Administrative Conference of the United States,3/1/2010,Present,Independent Agency
3,AA00,11,C,04,12,B,0560,2,99,GS-12,...,1,202403,1,99200.0,2.5,AA00,Administrative Conference of the United States,3/1/2010,Present,Independent Agency
4,AA00,11,D,04,13,B,0905,1,99,GS-13,...,1,202403,1,117962.0,2.5,AA00,Administrative Conference of the United States,3/1/2010,Present,Independent Agency


In [75]:
# Step 3
employment_summary_dept = fed_employment[['AGYSUB', 'Department']]
employment_summary_dept = employment_summary_dept.groupby('Department').size().reset_index(name='Number of Employees')
employment_summary_dept

,Department,Number of Employees
0,Consumer Financial Protection Bureau (CFPB),1687
1,Department of Agriculture (USDA),90664
2,Department of Commerce (DOC),54250
3,Department of Defense (DOD),757786
4,Department of Education (ED),3083
5,Department of Energy (DOE),15388
6,Department of Health and Human Services (HHS),98686
7,Department of Homeland Security (DHS),222173
8,Department of Housing and Urban Development (HUD),6443
9,Department of Justice (DOJ),123377


In [76]:
# Step 4
employment_summary_dept = employment_summary_dept.merge(budgets, left_on='Department', right_on='Agency', how='right')
employment_summary_dept

,Department,Number of Employees,Agency,Congressional Justification of Budget (CJ),Budget,Percent of Total Budget
0,Department of the Treasury (TREAS),112399.0,Department of the Treasury (TREAS),https://www.treasury.gov/cj,"$3,125,545,544,294",28.86%
1,Department of Health and Human Services (HHS),98686.0,Department of Health and Human Services (HHS),https://www.hhs.gov/cj,"$2,369,129,444,329",21.88%
2,Department of Defense (DOD),757786.0,Department of Defense (DOD),https://www.defense.gov/cj,"$1,713,980,313,483",15.83%
3,NaN,NaN,Social Security Administration (SSA),https://www.ssa.gov/cj,"$452,144,957,484",4.17%
4,Department of Veterans Affairs (VA),480579.0,Department of Veterans Affairs (VA),https://www.va.gov/cj,"$407,272,934,513",3.76%
...,...,...,...,...,...,...
103,NaN,NaN,John F. Kennedy Center for the Performing Arts...,https://www.kennedy-center.org/cj,$0,Less than 0.01%
104,NaN,NaN,Federal Financial Institutions Examination Cou...,not available,$0,Less than 0.01%
105,NaN,NaN,Delta Regional Authority (DRA),https://dra.gov/cj,$0,Less than 0.01%
106,NaN,NaN,Commission for the Preservation of America's H...,https://www.heritageabroad.gov/cj,$0,Less than 0.01%


In [79]:
employment_summary_dept.sort_values(by='Number of Employees', ascending=False).to_csv("../data summaries/agencies_budget_employees_summary.csv")